# Chapter 1: Rotation and SE(3) Basics

## 🎯 학습 목표

이 챕터에서는 Pose Graph Optimization의 핵심이 되는 회전과 SE(3) 변환의 기초를 배웁니다.

- 회전의 다양한 표현법 이해 (쿼터니언, 회전 행렬, 회전 벡터)
- SE(3) 그룹의 개념과 연산
- Tangent space와 Manifold의 관계
- 실제 SLAM에서의 활용

## 📚 이론적 배경

### 회전의 표현

3D 공간에서 회전을 표현하는 방법은 여러 가지가 있습니다:

1. **회전 행렬 (Rotation Matrix)**: $R \in SO(3)$, 3×3 직교 행렬
2. **쿼터니언 (Quaternion)**: $q = [q_x, q_y, q_z, q_w]$, 단위 쿼터니언
3. **회전 벡터 (Rotation Vector)**: $r = \theta \cdot \hat{n}$, 축-각도 표현
4. **오일러 각 (Euler Angles)**: Roll, Pitch, Yaw (우리는 사용하지 않음)

### SE(3) 그룹

SE(3)은 Special Euclidean Group으로, 3D 공간에서의 rigid transformation을 나타냅니다:

$$T = \begin{bmatrix} R & t \\ 0 & 1 \end{bmatrix} \in SE(3)$$

여기서 $R \in SO(3)$는 회전, $t \in \mathbb{R}^3$는 이동을 나타냅니다.

## 🔧 필요한 라이브러리 설치 및 임포트

In [ ]:
# 필요한 패키지들을 임포트합니다
import numpy as np
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 출력 포맷 설정
np.set_printoptions(precision=4, suppress=True)

print("✅ 모든 라이브러리가 성공적으로 임포트되었습니다!")

## 1. 회전 표현법 간 변환

먼저 nano-pgo에서 사용하는 기본 변환 함수들을 구현해봅시다.

In [ ]:
def quat_to_rotmat(qx, qy, qz, qw):
    """쿼터니언을 회전 행렬로 변환
    
    Args:
        qx, qy, qz, qw: 쿼터니언 성분 (스칼라 부분이 qw)
        
    Returns:
        R: 3x3 회전 행렬
    """
    rotation = Rotation.from_quat([qx, qy, qz, qw])
    return rotation.as_matrix()

def rotvec_to_quat(rotvec):
    """회전 벡터를 쿼터니언으로 변환
    
    Args:
        rotvec: 3차원 회전 벡터 (축 * 각도)
        
    Returns:
        q: [qx, qy, qz, qw] 쿼터니언
    """
    rotation = Rotation.from_rotvec(rotvec)
    q = rotation.as_quat()
    return q

def rotmat_to_rotvec(R):
    """회전 행렬을 회전 벡터로 변환
    
    Args:
        R: 3x3 회전 행렬
        
    Returns:
        rotvec: 3차원 회전 벡터
    """
    rotation = Rotation.from_matrix(R)
    rotvec = rotation.as_rotvec()
    return rotvec

def rotvec_to_rotmat(rotvec):
    """회전 벡터를 회전 행렬로 변환
    
    Args:
        rotvec: 3차원 회전 벡터
        
    Returns:
        R: 3x3 회전 행렬
    """
    rotation = Rotation.from_rotvec(rotvec)
    R = rotation.as_matrix()
    return R

print("✅ 기본 변환 함수들이 정의되었습니다!")

### 🧪 실습 1: 회전 표현법 변환 테스트

실제로 이 함수들이 어떻게 작동하는지 확인해봅시다.

In [ ]:
# 예제: Z축 기준 90도 회전
angle_deg = 90
angle_rad = np.deg2rad(angle_deg)

# 1. 회전 벡터로 시작
rotvec = np.array([0, 0, angle_rad])  # Z축 기준 90도
print(f"1. 회전 벡터 (Z축 {angle_deg}도): {rotvec}")

# 2. 회전 벡터 → 회전 행렬
R = rotvec_to_rotmat(rotvec)
print(f"\n2. 회전 행렬:\n{R}")

# 3. 회전 벡터 → 쿼터니언
q = rotvec_to_quat(rotvec)
print(f"\n3. 쿼터니언 [x,y,z,w]: {q}")

# 4. 쿼터니언 → 회전 행렬 (검증)
R_from_quat = quat_to_rotmat(q[0], q[1], q[2], q[3])
print(f"\n4. 쿼터니언에서 복원한 회전 행렬:\n{R_from_quat}")

# 5. 회전 행렬 → 회전 벡터 (검증)
rotvec_restored = rotmat_to_rotvec(R)
print(f"\n5. 복원된 회전 벡터: {rotvec_restored}")

# 검증
print(f"\n✅ 변환 정확도 검증:")
print(f"   회전 행렬 차이: {np.linalg.norm(R - R_from_quat):.6f}")
print(f"   회전 벡터 차이: {np.linalg.norm(rotvec - rotvec_restored):.6f}")

### 📊 시각화: 회전의 효과

회전이 3D 공간의 점들에 어떤 영향을 미치는지 시각화해봅시다.

In [ ]:
def visualize_rotation(R, title="Rotation Visualization"):
    """회전 행렬의 효과를 3D로 시각화"""
    fig = plt.figure(figsize=(12, 5))
    
    # 원본 좌표축
    axes_original = np.eye(3)
    # 회전된 좌표축
    axes_rotated = R @ axes_original
    
    # 큐브의 꼭짓점들
    cube_vertices = np.array([
        [0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0],  # 하단
        [0, 0, 1], [1, 0, 1], [1, 1, 1], [0, 1, 1]   # 상단
    ]).T
    
    # Subplot 1: 원본
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.set_title("Original")
    
    # 좌표축 그리기
    colors = ['r', 'g', 'b']
    labels = ['X', 'Y', 'Z']
    for i in range(3):
        ax1.quiver(0, 0, 0, axes_original[0, i], axes_original[1, i], axes_original[2, i],
                   color=colors[i], arrow_length_ratio=0.1, linewidth=3, label=labels[i])
    
    # 큐브 그리기
    ax1.scatter(cube_vertices[0], cube_vertices[1], cube_vertices[2], c='gray', s=50)
    
    # Subplot 2: 회전 후
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.set_title(f"After Rotation: {title}")
    
    # 회전된 좌표축
    for i in range(3):
        ax2.quiver(0, 0, 0, axes_rotated[0, i], axes_rotated[1, i], axes_rotated[2, i],
                   color=colors[i], arrow_length_ratio=0.1, linewidth=3, label=labels[i])
    
    # 회전된 큐브
    cube_rotated = R @ cube_vertices
    ax2.scatter(cube_rotated[0], cube_rotated[1], cube_rotated[2], c='gray', s=50)
    
    # 축 설정
    for ax in [ax1, ax2]:
        ax.set_xlim([-1.5, 1.5])
        ax.set_ylim([-1.5, 1.5])
        ax.set_zlim([-1.5, 1.5])
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.legend()
    
    plt.tight_layout()
    plt.show()

# 다양한 회전 시각화
rotations = [
    (np.array([0, 0, np.pi/2]), "Z축 90도"),
    (np.array([np.pi/2, 0, 0]), "X축 90도"),
    (np.array([0, np.pi/2, 0]), "Y축 90도"),
    (np.array([np.pi/4, np.pi/4, 0]), "X,Y축 각각 45도")
]

for rotvec, title in rotations[:1]:  # 첫 번째 예제만 표시
    R = rotvec_to_rotmat(rotvec)
    visualize_rotation(R, title)

## 2. SE(3) 변환

이제 회전과 이동을 함께 다루는 SE(3) 변환을 살펴봅시다.

In [ ]:
class SE3Transform:
    """SE(3) 변환을 다루는 클래스"""
    
    def __init__(self, R=None, t=None):
        """SE(3) 변환 초기화
        
        Args:
            R: 3x3 회전 행렬 (기본값: 단위 행렬)
            t: 3x1 이동 벡터 (기본값: 영벡터)
        """
        self.R = np.eye(3) if R is None else R
        self.t = np.zeros(3) if t is None else t
    
    def matrix(self):
        """4x4 동차 변환 행렬 반환"""
        T = np.eye(4)
        T[:3, :3] = self.R
        T[:3, 3] = self.t
        return T
    
    def inverse(self):
        """SE(3) 역변환 계산
        
        T^{-1} = [R^T  -R^T*t]
                 [0     1    ]
        """
        R_inv = self.R.T
        t_inv = -R_inv @ self.t
        return SE3Transform(R_inv, t_inv)
    
    def compose(self, other):
        """두 SE(3) 변환의 합성 (self * other)
        
        T1 * T2 = [R1*R2  R1*t2 + t1]
                  [0      1        ]
        """
        R_new = self.R @ other.R
        t_new = self.R @ other.t + self.t
        return SE3Transform(R_new, t_new)
    
    def transform_point(self, point):
        """3D 점을 변환"""
        return self.R @ point + self.t
    
    def __repr__(self):
        return f"SE3Transform:\nR:\n{self.R}\nt: {self.t}"

print("✅ SE3Transform 클래스가 정의되었습니다!")

### 🧪 실습 2: SE(3) 변환 연산

SE(3) 그룹의 주요 연산들을 실습해봅시다.

In [ ]:
# 1. 두 개의 SE(3) 변환 생성
# T1: Z축 기준 90도 회전 + (1, 0, 0) 이동
R1 = rotvec_to_rotmat(np.array([0, 0, np.pi/2]))
t1 = np.array([1, 0, 0])
T1 = SE3Transform(R1, t1)

# T2: X축 기준 90도 회전 + (0, 1, 0) 이동
R2 = rotvec_to_rotmat(np.array([np.pi/2, 0, 0]))
t2 = np.array([0, 1, 0])
T2 = SE3Transform(R2, t2)

print("T1 (Z축 90도 회전 + X방향 1 이동):")
print(T1)
print("\nT2 (X축 90도 회전 + Y방향 1 이동):")
print(T2)

# 2. 변환의 합성
T12 = T1.compose(T2)
print("\n\nT1 * T2 (합성 변환):")
print(T12)

# 3. 역변환
T1_inv = T1.inverse()
print("\n\nT1^{-1} (역변환):")
print(T1_inv)

# 4. 검증: T1 * T1^{-1} = I
identity_check = T1.compose(T1_inv)
print("\n\nT1 * T1^{-1} (단위 변환이어야 함):")
print(identity_check)
print(f"\n회전 부분 오차: {np.linalg.norm(identity_check.R - np.eye(3)):.6f}")
print(f"이동 부분 오차: {np.linalg.norm(identity_check.t):.6f}")

### 📊 시각화: SE(3) 변환의 연쇄 적용

로봇이 이동하면서 생기는 연속적인 SE(3) 변환을 시각화해봅시다.

In [ ]:
def visualize_se3_chain(transforms, labels=None):
    """SE(3) 변환의 연쇄를 시각화"""
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # 누적 변환 계산
    cumulative_transforms = [SE3Transform()]  # 항등 변환으로 시작
    for T in transforms:
        cumulative_transforms.append(cumulative_transforms[-1].compose(T))
    
    # 경로 그리기
    positions = np.array([T.t for T in cumulative_transforms])
    ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], 
            'b-', linewidth=2, label='Robot Path')
    
    # 각 위치에서의 좌표계 그리기
    colors = ['r', 'g', 'b']
    for i, T in enumerate(cumulative_transforms):
        # 위치
        pos = T.t
        ax.scatter(*pos, s=100, c='black')
        
        # 좌표축
        for j in range(3):
            axis = T.R[:, j] * 0.3  # 축 길이 조정
            ax.quiver(pos[0], pos[1], pos[2], 
                     axis[0], axis[1], axis[2],
                     color=colors[j], arrow_length_ratio=0.2)
        
        # 라벨
        if labels and i < len(labels):
            ax.text(pos[0], pos[1], pos[2] + 0.2, labels[i], fontsize=10)
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('SE(3) Transform Chain Visualization')
    ax.legend()
    
    # 축 범위 설정
    all_points = positions
    margin = 1.0
    ax.set_xlim([all_points[:, 0].min() - margin, all_points[:, 0].max() + margin])
    ax.set_ylim([all_points[:, 1].min() - margin, all_points[:, 1].max() + margin])
    ax.set_zlim([all_points[:, 2].min() - margin, all_points[:, 2].max() + margin])
    
    plt.show()

# 로봇 움직임 시뮬레이션
robot_movements = [
    SE3Transform(np.eye(3), np.array([1, 0, 0])),  # 전진
    SE3Transform(rotvec_to_rotmat(np.array([0, 0, np.pi/2])), np.array([0, 0, 0])),  # 좌회전
    SE3Transform(np.eye(3), np.array([1, 0, 0])),  # 전진
    SE3Transform(rotvec_to_rotmat(np.array([0, 0, np.pi/2])), np.array([0, 0, 0])),  # 좌회전
    SE3Transform(np.eye(3), np.array([1, 0, 0])),  # 전진
]

labels = ['Start', 'Move 1', 'Turn 1', 'Move 2', 'Turn 2', 'Move 3']

visualize_se3_chain(robot_movements, labels)

## 3. Tangent Space와 Manifold

SE(3)는 manifold이며, 최적화를 위해서는 tangent space에서 작업해야 합니다.

In [ ]:
def skew_symmetric(v):
    """3D 벡터를 skew-symmetric 행렬로 변환
    
    [v]_× = [ 0   -vz   vy]
            [ vz   0   -vx]
            [-vy   vx   0 ]
    """
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def se3_exp(xi):
    """se(3) → SE(3) exponential map
    
    Args:
        xi: 6D vector [translation, rotation]
        
    Returns:
        T: SE(3) transformation
    """
    v = xi[:3]  # translation part
    w = xi[3:]  # rotation part
    
    # Rotation
    theta = np.linalg.norm(w)
    if theta < 1e-6:
        R = np.eye(3) + skew_symmetric(w)
    else:
        R = rotvec_to_rotmat(w)
    
    # Translation (simplified for small angles)
    t = v
    
    return SE3Transform(R, t)

def se3_log(T):
    """SE(3) → se(3) logarithm map
    
    Args:
        T: SE3Transform object
        
    Returns:
        xi: 6D vector [translation, rotation]
    """
    # Rotation part
    w = rotmat_to_rotvec(T.R)
    
    # Translation part (simplified)
    v = T.t
    
    return np.concatenate([v, w])

print("✅ Exponential과 Logarithm map이 정의되었습니다!")

### 🧪 실습 3: Tangent Space에서의 연산

최적화에서 중요한 tangent space 연산을 실습해봅시다.

In [ ]:
# 1. SE(3) 변환을 tangent vector로 변환
T = SE3Transform(
    rotvec_to_rotmat(np.array([0.1, 0.2, 0.3])),
    np.array([1, 2, 3])
)

print("원본 SE(3) 변환:")
print(T)

# 2. Log map: SE(3) → se(3)
xi = se3_log(T)
print(f"\nTangent vector (se(3)): {xi}")
print(f"  - Translation part: {xi[:3]}")
print(f"  - Rotation part: {xi[3:]}")

# 3. Exp map: se(3) → SE(3)
T_restored = se3_exp(xi)
print("\n복원된 SE(3) 변환:")
print(T_restored)

# 4. 오차 검증
print("\n✅ 변환 정확도:")
print(f"  - 회전 오차: {np.linalg.norm(T.R - T_restored.R):.6f}")
print(f"  - 이동 오차: {np.linalg.norm(T.t - T_restored.t):.6f}")

## 4. SLAM에서의 실제 응용: Relative Pose Error

Pose Graph Optimization에서 핵심이 되는 relative pose error를 계산해봅시다.

In [ ]:
def compute_relative_pose_error(Ti, Tj, Tij_measured):
    """두 포즈 간의 relative pose error 계산
    
    에러는 다음과 같이 정의됩니다:
    T_error = T_ij_measured^{-1} * T_i^{-1} * T_j
    
    Args:
        Ti, Tj: 포즈 i와 j의 SE(3) 변환
        Tij_measured: 측정된 상대 변환
        
    Returns:
        error_vector: 6D error vector [translation, rotation]
    """
    # 예측된 상대 변환
    Tij_predicted = Ti.inverse().compose(Tj)
    
    # 에러 변환
    T_error = Tij_measured.inverse().compose(Tij_predicted)
    
    # Tangent space로 변환
    error_vector = se3_log(T_error)
    
    return error_vector, T_error

# 예제: 두 포즈와 측정값
# 포즈 i: 원점
Ti = SE3Transform()

# 포즈 j: 실제로는 (2, 1, 0)에 있고 30도 회전
Tj = SE3Transform(
    rotvec_to_rotmat(np.array([0, 0, np.pi/6])),
    np.array([2, 1, 0])
)

# 측정값: 약간의 노이즈가 있음
Tij_measured = SE3Transform(
    rotvec_to_rotmat(np.array([0, 0, np.pi/6 + 0.05])),  # 회전에 노이즈
    np.array([2.1, 0.9, 0])  # 이동에 노이즈
)

# 에러 계산
error_vector, T_error = compute_relative_pose_error(Ti, Tj, Tij_measured)

print("포즈 간 상대 에러 분석:")
print(f"\n에러 벡터 (6D): {error_vector}")
print(f"  - Translation error: {error_vector[:3]}")
print(f"  - Rotation error: {error_vector[3:]}")
print(f"\n총 에러 크기: {np.linalg.norm(error_vector):.4f}")

## 5. 요약 및 핵심 포인트

### 🎓 이 챕터에서 배운 내용:

1. **회전 표현법**
   - 쿼터니언, 회전 행렬, 회전 벡터 간 변환
   - 각 표현법의 장단점과 사용 시기

2. **SE(3) 그룹**
   - 3D rigid transformation의 수학적 표현
   - 합성(composition)과 역변환(inverse) 연산

3. **Tangent Space**
   - Manifold 상에서의 최적화를 위한 필수 개념
   - Exponential과 Logarithm map

4. **SLAM 응용**
   - Relative pose error의 정의와 계산
   - Pose Graph Optimization의 기초

### 💡 다음 챕터 예고:

다음 챕터에서는 g2o 파일 포맷을 이해하고, 실제 SLAM 데이터를 로드하여 시각화하는 방법을 배웁니다.

## 🏋️ 연습 문제

### 문제 1: 회전 합성
두 회전 R1(X축 45도)과 R2(Y축 30도)를 순서대로 적용했을 때의 최종 회전을 계산하고, 역순으로 적용했을 때와 비교해보세요.

### 문제 2: SE(3) 보간
두 SE(3) 변환 사이를 선형 보간하는 함수를 작성해보세요. (힌트: tangent space에서 보간)

### 문제 3: 에러 민감도
측정 노이즈의 크기가 relative pose error에 미치는 영향을 분석해보세요.

In [ ]:
# 여기에 연습 문제를 풀어보세요!
# 예제 코드:

# 문제 1 시작
# R1 = rotvec_to_rotmat(np.array([np.pi/4, 0, 0]))  # X축 45도
# R2 = rotvec_to_rotmat(np.array([0, np.pi/6, 0]))  # Y축 30도
# ...
